In [2]:
# Import libraries
import cv2
import csv
from datetime import datetime
import mediapipe as mp

#from mediapipe.tasks import python
#from mediapipe.tasks.python import vision
# from google.colab.patches import cv2_imshow
# import numpy as np
# import matplotlib.pyplot as plt

In [19]:
# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Initialize video capture
cap = cv2.VideoCapture(0)

# Set the FPS
cap.set(cv2.CAP_PROP_FPS, 60)

# Indices for the nose tip and chin landmarks
NOSE_TIP_INDEX = 1
CHIN_INDEX = 152

# Set drawing specifications
landmark_specs = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)

# Create a CSV file with a timestamp in the filename
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_filename = f"data/csv/face_landmarks_{timestamp}.csv"

with open(csv_filename, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Frame', 'Nose_Tip_X', 'Nose_Tip_Y', 'Chin_X', 'Chin_Y'])

    # Create Face Mesh object with specific settings
    with mp_face_mesh.FaceMesh(
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    ) as face_mesh:

        # Check if the video source is available
        if not cap.isOpened():
            print("Error: Could not open video source.")
        else:
            frame_count = 0
            # Process each video frame
            while cap.isOpened():
                success, image = cap.read()

                if not success:
                    print("Ignoring empty camera frame.")
                    continue

                # Convert the image to RGB for face mesh processing
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = face_mesh.process(image_rgb)

                # If face landmarks are detected
                if results.multi_face_landmarks:
                    for face_landmarks in results.multi_face_landmarks:
                        # Extract the landmark coordinates
                        nose_tip = face_landmarks.landmark[NOSE_TIP_INDEX]
                        chin = face_landmarks.landmark[CHIN_INDEX]

                        # Convert normalized coordinates to pixel values
                        h, w, _ = image.shape
                        nose_tip_coords = (int(nose_tip.x * w), int(nose_tip.y * h))
                        chin_coords = (int(chin.x * w), int(chin.y * h))

                        # Write coordinates to CSV
                        csv_writer.writerow([
                            frame_count,
                            nose_tip_coords[0], nose_tip_coords[1],
                            chin_coords[0], chin_coords[1],
                        ])

                        # Draw the landmark points on the image
                        for coords in [nose_tip_coords, chin_coords]:
                            cv2.circle(image, coords, 5, (0, 255, 0), -1)

                        # Draw lines between the points
                        cv2.line(image, nose_tip_coords, chin_coords, (255, 0, 0), 2)

                # Display the output
                cv2.imshow("Face Landmarks Detection", cv2.flip(image, 1))

                frame_count += 1

                # Break the loop if 'q' is pressed
                if cv2.waitKey(16) == ord('q'):
                    break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

print(f"Landmarks data saved to {csv_filename}")

Landmarks data saved to data/csv/face_landmarks_20241016_214323.csv
